In [ ]:
import numpy as np
import sklearn as sk
import sklearn.datasets as sk_ds
import sklearn.svm as sk_svm
import matplotlib.pyplot as plt

In [ ]:
wbc_features, wbc_targets = sk_ds.load_breast_cancer(True)
print('WBC: Zbiór {} danych z {} cechami'.format(len(wbc_targets),
                                                 wbc_features.shape[1]))

## SVM

### Klasyfikator liniowy

$$ y = f(\vec{w} \cdot \vec{x}) = f(\sum_i w_i x_i)  $$

$\vec{w}$ - wektor wag klasyfikatora

$\vec{x}$ - wektor wejściowy cech

$f$ - funkcja decyzyjna $\Re \to N $ odwzorowuje zbiór liczb rzeczywistych na klasę 

Dla dwóch klas funkcja $f$ zazwyczaj przypisuje iloczyn skalarny do jednej klasy jeśli jest on większy od pewnej wartości i do drugiej klasy jeśli nie jest. W takim przypadku klasyfikator liniowy dzieli przestrzeń wejściową za pomocą hiperpłaszczyzny, czyli uogólnienia płaszczyzny do n-wymiarów.

In [ ]:
def make_linear():
  in_features = wbc_features[40:70,:2]
  out_targets = wbc_targets[40:70]
  
  min_x = in_features[:,0].min() - 1
  max_x = in_features[:,0].max() + 1
  min_y = in_features[:,1].min() - 1
  max_y = in_features[:,1].max() + 1
  xx, yy = np.meshgrid(np.arange(min_x, max_x, 0.01),
                       np.arange(min_y, max_y, 0.01))
  classifier = sk_svm.LinearSVC(dual=False, tol=0.0000001)
  classifier.fit(in_features, out_targets)
  Z = classifier.predict(np.c_[xx.ravel(), yy.ravel()])
  Z = Z.reshape(xx.shape)
  plt.contourf(xx, yy, Z, cmap=plt.cm.coolwarm, alpha=0.8)
  plt.scatter(in_features[:,0], in_features[:,1], c=out_targets, cmap=plt.cm.coolwarm)
  plt.show()
  
make_linear()

### Klasyfikator SVM

### Trik kernelowy

### Jakość klasyfikacji

#### ROC - Receiver Operating Characteristic 
Krzywa ROC służy do oceny jakości klasyfikatora. Przedstawia zależność ilości prawdziwie pozytywnych ($T_p$) i fałszywie pozytywnych wyników ($T_f$). Krzywa idealnego klasyfikatora obejmuje lewy górny punkt wykresu (100% prawdziwie pozytywnych wyników i 0% fałszywie pozytywnych), zatem zazwyczaj większe pole pod wykresem jest lepsze. Istotne jest także nachylenie wykresu, bardziej stromy jest lepszy.

In [ ]:
from sklearn import datasets
from sklearn.datasets import make_classification
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt

bc = datasets.load_breast_cancer()
X = bc.data
y = bc.target

indices = y < 2
X_train, X_test, y_train, y_test = train_test_split(X[indices], y[indices], test_size=0.5, random_state=2)

classifier = KNeighborsClassifier(n_neighbors=3)
classifier.fit(X_train, y_train)

probs = classifier.predict_proba(X_test)
# keep probabilities for the positive outcome only
probs = probs[:, 1]

area_under_curve = roc_auc_score(y_test, probs)
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, probs)

plt.plot([0, 1], [0, 1], linestyle='--', label='no skill')
plt.plot(false_positive_rate, true_positive_rate, marker='.', label='ROC curve (area = {0:.2f})'.format(area_under_curve))

plt.legend(loc="lower right")
plt.xlabel('T_f (false positive rate)')
plt.ylabel('T_p (true positive rate)')
plt.show()

#### Precision-recall
Miara precision-recall służy do określania jakości klasyfikatora, szczególnie jeżeli liczebność obu klas znacznie się różni. Składa się z dwóch parametrów:
* **precision** ($P$): stosunek liczby prawdziwie pozytywnych wskazań ($T_p$) do liczby prawdziwie pozytywnych i fałszywie pozytywnych wskazań klasyfikatora ($T_p + T_f$)
$$ P = \frac{T_p}{T_p+T_f}$$
* **recall** ($R$): stosunek liczby prawdziwie pozytywnych wskazań ($T_p$) do liczby prawdziwie pozytywnych i fałszywie negatywnych wskazań klasyfikatora ($T_p + F_n$)
$$ R = \frac{T_p}{T_p+F_n}$$
Obie wartości zależą od progu klasyfikatora.

Zmniejszanie progu może zwiększyć ilość zwróconych wyników ($T_p+T_f$). Jeżeli próg był ustawiony zbyt wysoko, to dodatkowe wyniki będą prawdziwie pozytywne, dzięki czemu zwiększy się precision i recall. Jeżeli próg był odpowiedni lub zbyt niski, to w dodatkowych wynikach będzie dużo fałszywie pozytywnych punktów, przez co spadnie precision, a recall niewiele się zwiększy. Obrazuje to kształt wykresu:

In [ ]:
from sklearn.datasets import make_classification
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import auc
from sklearn.metrics import average_precision_score
import matplotlib.pyplot as plt

bc = datasets.load_breast_cancer()
X = bc.data
y = bc.target
# split into train/test sets, selecting only 2 classes
indices = y < 2
X_train, X_test, y_train, y_test = train_test_split(X[indices], y[indices], test_size=0.5, random_state=2)


classifier = KNeighborsClassifier(n_neighbors=3)
classifier.fit(X_train, y_train)
# predict probabilities
probs = classifier.predict_proba(X_test)
# keep probabilities for the positive outcome only
probs = probs[:, 1]
# predict class values
yhat = classifier.predict(X_test)
precision, recall, thresholds = precision_recall_curve(y_test, probs)
f1 = f1_score(y_test, yhat)
area_under_curve = auc(recall, precision)
average_precision = average_precision_score(y_test, probs)

print('f1=%.3f auc=%.3f ap=%.3f' % (f1, area_under_curve, average_precision))

plt.plot([0, 1], [0.5, 0.5], linestyle='--', label='no skill')
plt.plot(recall, precision, marker='.', label='precision-recall curve (area = {0:.2f})'.format(area_under_curve))

plt.legend(loc="best")
plt.xlabel('R (recall)')
plt.ylabel('P (precision)')
plt.show()

#### F1
F1 to miara jakości klasyfikatora. Jest średnią harmoniczną wartości precision i recall. Najlepszy klasyfikator ma F1 równy 1, a najgorszy 0.
$$F1 = (\frac{P^{-1} + R^{-1}}{2})^{-1} = 2\frac{P\cdotR}{P+}$$

#### Różnica między krzywymi ROC i precision-recall


In [ ]:
from sklearn.datasets import make_classification
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import auc
from sklearn.metrics import average_precision_score
import matplotlib.pyplot as pyplot
# generate 2 class dataset
X, y = make_classification(n_samples=1000, n_classes=2, weights=[0.9,0.1], random_state=1)
# split into train/test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=2)
# fit a model
model = KNeighborsClassifier(n_neighbors=3)
model.fit(X_train, y_train)
# predict probabilities
probs = model.predict_proba(X_test)
# keep probabilities for the positive outcome only
probs = probs[:, 1]
# calculate AUC
roc_auc = roc_auc_score(y_test, probs)
print('area under ROC curve: %.3f' % roc_auc)
# calculate roc curve
false_positive_ratio, true_positive_ratio, thresholds = roc_curve(y_test, probs)
plt.plot([0, 1], [0, 1], linestyle='--', label='no skill')
plt.plot(false_positive_ratio, true_positive_ratio, marker='.', label='ROC curve')
plt.legend(loc='best')
plt.xlabel('T_f (false positive rate)')
plt.ylabel('T_p (true positive rate)')
plt.show()

# predict class values
yhat = model.predict(X_test)

precision, recall, thresholds = precision_recall_curve(y_test, probs)
f1 = f1_score(y_test, yhat)
pr_auc = auc(recall, precision)
ap = average_precision_score(y_test, probs)
print('f1=%.3f auc=%.3f ap=%.3f' % (f1, pr_auc, ap))

plt.plot([0, 1], [0.1, 0.1], linestyle='--', label='no skill')
plt.plot(recall, precision, marker='.', label='precision-recall curve (area = {0:.2f})'.format(area_under_curve))
plt.legend(loc="lower right")
plt.xlabel('R (recall)')
plt.ylabel('P (precision)')
plt.show()

## Działanie SVM